## Deep Learning Move Convolutional Engine

By Evan Richardson

This notebook is dedicated to producing the move-selecting engine for the move prediction engine.  These models are used to select the location to move to, with each model corresponding to a particular piece.

In [1]:
import tensorflow
print(tensorflow.__version__)

import numpy as np
import pandas as pd
# from google.colab import drive
from sklearn.model_selection import train_test_split
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten

1.13.0-rc1


In [2]:
# drive.mount('/content/gdrive')
df = pd.read_csv("../data/2018_games_dl_pred.csv")

In [3]:
df.head()

Result  Move  Color  Piece  CurrentFile  CurrentRank  NewFile  NewRank  \
0       1     1      1      1            4            1        4        3   
1       1     2      1      1            5            1        5        3   
2       1     3      1      3            6            0        5        2   
3       1     4      1      1            3            1        3        2   
4       1     5      1      1            4            3        4        4   

    a1   b1 ...    g7   h7   a8   b8    c8    d8    e8    f8   g8   h8  
0  2.0  3.0 ...   7.0  7.0  8.0  9.0  10.0  11.0  12.0  10.0  9.0  8.0  
1  2.0  3.0 ...   7.0  7.0  8.0  9.0  10.0  11.0  12.0  10.0  9.0  8.0  
2  2.0  3.0 ...   7.0  7.0  8.0  9.0  10.0  11.0  12.0  10.0  9.0  8.0  
3  2.0  3.0 ...   7.0  7.0  8.0  NaN  10.0  11.0  12.0  10.0  9.0  8.0  
4  2.0  3.0 ...   7.0  7.0  8.0  NaN  10.0  11.0  12.0  10.0  9.0  8.0  

[5 rows x 72 columns]

In [4]:
df.shape

(2863632, 72)

In [5]:
pawn_moves = df.loc[df['Piece'] == 1]
rook_moves = df.loc[df['Piece'] == 2]
knight_moves = df.loc[df['Piece'] == 3]
bishop_moves = df.loc[df['Piece'] == 4]
queen_moves = df.loc[df['Piece'] == 5]
king_moves = df.loc[df['Piece'] == 6]

print(pawn_moves.shape, rook_moves.shape, knight_moves.shape, bishop_moves.shape, queen_moves.shape, king_moves.shape)

(774689, 72) (468141, 72) (504619, 72) (466076, 72) (349035, 72) (301072, 72)


In [6]:
def translateGameStates(df):
  states = []

  for position in df.values:
    gameState = np.zeros((8, 8, 6))
    squares = position[-64:]

    for rank in range(8):
      for file in range(8):
        index = rank * 8 + file
        piece = squares[index]

        if piece > 6:
          gameState[rank][file][int(piece) - 7] = -1
        elif piece > 0:
          gameState[rank][file][int(piece) - 1] = 1

    states.append(gameState)

  return np.array(states)

The input "image" that is created to be consumed by the network is an 8x8 representation of the board with 6 output channels, with a 1 representing a friendly piece (in this model's case only white), and a -1 representing an enemy piece (black).  The 6 channels also correspond to a particular piece.

In [7]:
def translateOutputSet(df, file_col, rank_col):
  states = []

  for position in df.values:
    state = np.zeros(64)

    file = position[file_col]
    rank = position[rank_col]

    state[int(rank) * 8 + int(file)] = 1
    states.append(state)

  return np.array(states)

The output shape represents an 8x8 board, with a 1 representing the selected move.  For use with a dense layer, the board is flattened to a 64 length array.

In [8]:
def createTrainAndTestSets(df):
  train, test = train_test_split(df)
  print(df.shape, train.shape, test.shape)
  
  x_train = translateGameStates(train)
  x_test = translateGameStates(test)

  y_train = translateOutputSet(train, 6, 7)
  y_test = translateOutputSet(test, 6, 7)
  
  return (x_train, x_test, y_train, y_test)

In [9]:
pawn_x_train, pawn_x_test, pawn_y_train, pawn_y_test = createTrainAndTestSets(pawn_moves)
rook_x_train, rook_x_test, rook_y_train, rook_y_test = createTrainAndTestSets(rook_moves)
knight_x_train, knight_x_test, knight_y_train, knight_y_test = createTrainAndTestSets(knight_moves)
bishop_x_train, bishop_x_test, bishop_y_train, bishop_y_test = createTrainAndTestSets(bishop_moves)
queen_x_train, queen_x_test, queen_y_train, queen_y_test = createTrainAndTestSets(queen_moves)
king_x_train, king_x_test, king_y_train, king_y_test = createTrainAndTestSets(king_moves)

(774689, 72) (581016, 72) (193673, 72)
(468141, 72) (351105, 72) (117036, 72)
(504619, 72) (378464, 72) (126155, 72)
(466076, 72) (349557, 72) (116519, 72)
(349035, 72) (261776, 72) (87259, 72)
(301072, 72) (225804, 72) (75268, 72)


In [10]:
input_shape = (8, 8, 6)
print(input_shape)

(8, 8, 6)


In [11]:
def train_model(x_train, x_test, y_train, y_test):
  batch_size = 250
  num_classes = 64
  epochs = 15

  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(Flatten())
  model.add(Dense(num_classes, activation='softmax'))

  model.summary()

  model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                optimizer='adam',
                metrics=['accuracy'])

  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  
  return model

In [12]:
pawn_model = train_model(pawn_x_train, pawn_x_test, pawn_y_train, pawn_y_test)
rook_model = train_model(rook_x_train, rook_x_test, rook_y_train, rook_y_test)
knight_model = train_model(knight_x_train, knight_x_test, knight_y_train, knight_y_test)
bishop_model = train_model(bishop_x_train, bishop_x_test, bishop_y_train, bishop_y_test)
queen_model = train_model(queen_x_train, queen_x_test, queen_y_train, queen_y_test)
king_model = train_model(king_x_train, king_x_test, king_y_train, king_y_test)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 6, 32)          1760      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 128)         36992     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
Total params: 169,888
Trainable params: 169,888
Non-trainable params: 0
_________________________________________________________________
Train on 581016 samples, validate on 193673 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
581016/581016 [==============================] - 50s 86us/sample - loss: 1.4261 - acc

Epoch 1/15
225804/225804 [==============================] - 20s 90us/sample - loss: 1.5796 - acc: 0.4382 - val_loss: 1.1625 - val_acc: 0.5448
Epoch 2/15
225804/225804 [==============================] - 17s 74us/sample - loss: 1.0935 - acc: 0.5686 - val_loss: 1.0762 - val_acc: 0.5717
Epoch 3/15
225804/225804 [==============================] - 17s 73us/sample - loss: 1.0037 - acc: 0.6023 - val_loss: 1.0144 - val_acc: 0.5951
Epoch 4/15
225804/225804 [==============================] - 17s 76us/sample - loss: 0.9490 - acc: 0.6234 - val_loss: 0.9946 - val_acc: 0.6041
Epoch 5/15
225804/225804 [==============================] - 17s 75us/sample - loss: 0.9090 - acc: 0.6396 - val_loss: 0.9841 - val_acc: 0.6104
Epoch 6/15
225804/225804 [==============================] - 17s 74us/sample - loss: 0.8760 - acc: 0.6540 - val_loss: 0.9744 - val_acc: 0.6150
Epoch 7/15
225804/225804 [==============================] - 17s 74us/sample - loss: 0.8479 - acc: 0.6640 - val_loss: 0.9699 - val_acc: 0.6185
Epoch 

In [0]:
bad = []
for i in range(len(pawn_x_test)):
  a = np.array([pawn_x_test[i]])
  b = pawn_model.predict(a)[0]
  
  index = np.argmax(b)
  rank = int(index / 8)
  file = index % 8
  
  square = pawn_x_test[i][rank][file]
  if 1 in square:
    bad.append(i)
    print(i, bad)

In [13]:
pawn_model.save('../models/20190213_pawn_convolution.h5')
rook_model.save('../models/20190213_rook_convolution.h5')
knight_model.save('../models/20190213_knight_convolution.h5')
bishop_model.save('../models/20190213_bishop_convolution.h5')
queen_model.save('../models/20190213_queen_convolution.h5')
king_model.save('../models/20190213_king_convolution.h5')

With these models trained from the extensive datasets, it will always select a valid move, and does well throughout the game, with the only exception being that the endgame sometimes gets repetitive.

![Example 1](Example6.PNG)

Here is also an example of a midgame against the level 3 engine on chess.com

![Example 1](Example7.PNG)

Using the model I have been able to defeat the level 3 engine, but did run into an issue where the moves would get kind of repetitive.

![Example 1](Example8.PNG)

Overall though I would still consider this model a success even if some moves didn't work out correctly.